In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim

# Importaciones locales
from dataset import SongDataset
from model import initialize_model
from train import train, validate
import time
import plotly.express as px

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [3]:
# Configuración
DATA_PATH = "../data"
MODEL_NAME = "gpt2-medium"
BATCH_SIZE = 8
EPOCHS = 100
MAX_LENGTH = 200
LEARNING_RATE = 2e-5

# Cargar el dataset
train_dataset = SongDataset(f"{DATA_PATH}/train.csv", MODEL_NAME, MAX_LENGTH)
val_dataset = SongDataset(f"{DATA_PATH}/validation.csv", MODEL_NAME, MAX_LENGTH)


# Crear dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


In [4]:
# Inicializar el modelo y dispositivo
model, device = initialize_model(MODEL_NAME, 5)  # Suponiendo clasificación binaria

# Definir optimizador y función de pérdida
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()


In [5]:
train_losses = []
validation_losses = []

In [6]:
print(f"{'EPOCH':^10}|{'TRAIN-LOSS':^14}|{'VAL-LOSS':^12}|{'TIME':^8}|{'BEST':^9}")
print("=" * 56)

# Inicializar el mejor MSE y el mejor modelo
best_validation_loss = float("inf")
best_model = None


for epoch in range(EPOCHS):
    ti = time.time()

    train_loss = train(model, train_dataloader, optimizer, criterion, device)
    val_loss = validate(model, val_dataloader, criterion, device)

    train_losses.append(train_loss)
    validation_losses.append(val_loss)

    # Comprobar si el MSE de validación actual es mejor que el mejor MSE anterior
    best = ""
    if val_loss < best_validation_loss:
        best_validation_loss = val_loss
        best_model = model.state_dict()
        torch.save(best_model, "../output_models/best_model.pt")  # Guardar el mejor modelo en un archivo
        best = "***"

    # Imprimir los resultados en una tabla
    print(f"{epoch + 1:^10}|{train_loss:^14.2f}|{val_loss:^12.2f}|{time.time()-ti:^8.2f}|{best:^9}")


  EPOCH   |  TRAIN-LOSS  |  VAL-LOSS  |  TIME  |  BEST   


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
fig = px.line(x=range(len(validation_losses)), y=validation_losses, markers=True)

fig.update_traces(
    marker=dict(color="red", size=7, symbol="x"),
    line=dict(color="blue", width=3),
)

fig.update_layout(
    coloraxis_showscale=False,
    height=400,
    width=600,
    xaxis={"title": "epoch"},
    yaxis={"title": "val-accuracy"},
    title="CrossEntropyLoss vs. No. of epochs",
    template="ggplot2",
)
fig.show()


In [ ]:
fig = px.line(
    x=range(len(validation_losses)),
    y=[train_losses, validation_losses],
    markers=True,
)

new_legend_names = {
    "wide_variable_0": "Training",
    "wide_variable_1": "Validation",
}

fig.for_each_trace(
    lambda t: t.update(
        name=new_legend_names[t.name],
        legendgroup=new_legend_names[t.name],
        hovertemplate=t.hovertemplate.replace(t.name, new_legend_names[t.name]),
    )
)

fig.update_layout(
    coloraxis_showscale=False,
    height=400,
    width=600,
    xaxis={"title": "epoch"},
    yaxis={"title": "loss"},
    title="Loss vs. No. of epochs",
    template="ggplot2",
    legend_title=None,
)
fig.show()


In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()
